In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinkoff/stories_description.csv
/kaggle/input/tinkoff/sample_submit.csv
/kaggle/input/tinkoff/transactions.csv
/kaggle/input/tinkoff/customer_test.csv
/kaggle/input/tinkoff/Hack_desc.xlsx
/kaggle/input/tinkoff/customer_train.csv
/kaggle/input/tinkoff/stories_reaction_test.csv
/kaggle/input/tinkoff/stories_reaction_train.csv


In [2]:
import numpy as np
import pandas as pd

In [3]:
stories_desc = pd.read_csv('/kaggle/input/tinkoff/stories_description.csv')
sample_submit = pd.read_csv('/kaggle/input/tinkoff/sample_submit.csv')
transactions = pd.read_csv('/kaggle/input/tinkoff/transactions.csv')
test_cus = pd.read_csv('/kaggle/input/tinkoff/customer_test.csv')
train_cus = pd.read_csv('/kaggle/input/tinkoff/customer_train.csv')

1. Customers train and test preprocessing

In [4]:
def process(cust):
    mar_statuses = cust['marital_status_cd'].unique()
    mar2 = dict(zip(mar_statuses, range(len(mar_statuses))))

    cust['marital_status_cd'] = cust['marital_status_cd'].map(mar2)
    cust['gender_cd'] = cust['gender_cd'].map({'M': 0, 'F': 1})
    cust['age'] = cust['age'].fillna(cust['age'].median())
    cust['children_cnt'] = cust['children_cnt'].fillna(-1)
    cust['gender_cd'] = cust['gender_cd'].fillna(-1)

    
    for i in range(7):
        cust[f'product_{i}'] = cust[f'product_{i}'].fillna('NAN')
        cust[f'product_{i}'] = cust[f'product_{i}'].map({'OPN': 1, 'UTL': 2, 'CLS': 3, 'NAN': 4})
    
    return cust

cus_train = process(train_cus)
cus_test = process(test_cus)
cus_train.head(50)

,customer_id,product_0,product_1,product_2,product_3,product_4,product_5,product_6,gender_cd,age,marital_status_cd,children_cnt,first_session_dttm,job_position_cd,job_title
0,894436,4,4,4,4,4,2,4,0.0,30.0,0,0.0,2018-03-20 09:10:16,1,Неруководящий сотрудник - обсл. Персонал
1,524526,4,2,4,4,4,2,4,1.0,20.0,1,0.0,2017-03-29 20:38:45,16,NaN
2,498134,4,2,4,4,4,4,4,1.0,25.0,1,0.0,2018-03-12 11:25:06,22,NaN
3,278941,4,4,2,3,4,2,2,0.0,25.0,2,-1.0,2016-02-21 18:47:51,16,Неруководящий сотрудник - специалист
4,877312,4,2,4,4,4,4,4,1.0,40.0,0,0.0,2018-03-07 11:17:02,22,NaN
5,821806,4,4,4,2,4,2,4,1.0,25.0,2,0.0,2018-01-15 09:23:54,22,NaN
6,782728,4,2,4,4,4,4,4,0.0,35.0,0,0.0,2017-12-26 09:02:58,22,NaN
7,540071,4,4,4,4,4,2,4,0.0,30.0,2,0.0,2018-05-07 16:05:38,11,NaN
8,66158,4,2,4,4,4,2,4,1.0,30.0,0,0.0,2013-03-17 08:26:13,22,СПЕЦИАЛИСТ
9,804202,4,4,4,2,4,2,4,0.0,20.0,2,0.0,2017-12-29 14:32:49,11,NaN


In [5]:
cus_train = cus_train.dropna(subset=['first_session_dttm'])
cus_test = cus_test.dropna(subset=['first_session_dttm'])
cus_train.drop(['job_title'], axis = 1, inplace=True)
cus_test.drop(['job_title'], axis = 1, inplace=True)

print(cus_train.shape)
print(cus_test.shape)

(49997, 14)
(54239, 14)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
now = pd.to_datetime('now')


In [7]:
now = pd.Timestamp('now')
cus_train['first_ses'] = pd.to_datetime(cus_train['first_session_dttm'])
cus_test['first_ses'] = pd.to_datetime(cus_test['first_session_dttm'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
cus_train['months_after_first_session']=((now - cus_train['first_ses']).dt.total_seconds() / (60*60*24*12)-12)
cus_test['months_after_first_session']=((now - cus_test['first_ses']).dt.total_seconds() / (60*60*24*12)-12)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


2. Transactions processing

In [9]:
#We have transactions made in May-July
transactions.transaction_month.min() #5
transactions.transaction_month.max() #7

7

In [10]:
def process_trans(data):
    if data['merchant_mcc'] in [5122, 5292, 5295, 5912]:
        return 'Pharmacy'
    if data['merchant_mcc'] in [1520, 1711, 1731, 1740, 1750, 1761, 1771, 1799, 2791, 2842, 5261, 
                        5021, 5039, 5046, 5051, 5065, 5072, 5074, 5085, 5198, 5200, 5211, 5231, 5251, 
                        5415, 5712, 5713, 5714, 5718, 5719, 5722, 7622, 7623, 7629, 7641, 7692, 7699]:
        return 'House, Repair'
    if data['merchant_mcc'] in [3351, 3352, 3353, 3354, 3355, 3356, 3357, 3358, 3359, 3360, 3361, 3362, 3363, 
                        3365, 3366, 3367, 3368, 3369, 3370, 3371, 3372, 3373, 3374, 3375, 3376, 3377, 3378, 
                        3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 
                        3393, 3394, 3395, 3396, 3397, 3398, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 
                        3408, 3409, 3410, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3422, 
                        3423, 3425, 3426, 3427, 3428, 3429, 3430, 3431, 3432, 3433, 3434, 3435, 3436, 3437, 
                        3438, 3439, 3441, 7512, 7513, 7519, 3364]:
        return 'Car Rental'
    if data['merchant_mcc'] in [5511, 5521, 5531, 5532, 5533, 5571, 7012, 7531, 7534, 7535, 7538, 7542, 7549]:
        return 'Auto Service'
    if data['merchant_mcc'] in [5309]:
        return 'Duty Free'
    if data['merchant_mcc'] in [4011, 4112]:
        return 'Train Tickets'
    if data['merchant_mcc'] in [5995]:
        return 'Animals'
    if data['merchant_mcc'] in [5932, 5937, 5970, 5971, 5972, 5973, 7829, 7832, 7841,5733, 5735]:
        return 'Art, Cinema, Music'
    if data['merchant_mcc'] in [5977, 7230, 7297, 7298]:
        return 'Beauty'
    if data['merchant_mcc'] in [5094, 5137, 5139, 5611, 5621, 5631, 5641, 5651, 5661, 5681, 5691, 5697, 
                                5698, 5699, 5931, 5944, 5949, 5950, 7296, 7631]:
        return 'Clothing and shoes'
    if data['merchant_mcc'] in [7911, 7922, 7929, 7932, 7933, 7941, 7991, 7992, 7993, 7994, 7996, 7997, 
                                7998, 7999, 8664]:
        return 'Entertainments'
    if data['merchant_mcc'] in [5811, 5812, 5813]:
        return 'Restaurants'
    if data['merchant_mcc'] in [5655, 5940, 5941]:
        return 'Sporting goods'
    if data['merchant_mcc'] in [5947]:
        return 'Souvenirs'
    if data['merchant_mcc'] in [5297, 5298, 5300, 5411, 5412, 5422, 5441, 5451, 5462, 5499, 5715, 5921]:
        return 'Supermarkets'
    if data['merchant_mcc'] in [5172, 5541, 5542, 5983]:
        return 'Fuel'
    if data['merchant_mcc'] in [4111, 4121, 4131, 4457, 4468, 4784, 4789, 5013, 4121,
                                5271, 5551, 5561, 5592, 5598, 5599, 7511, 7523, 4121, 7512]:
        return 'Transport'
    if data['merchant_mcc'] in [5044, 5045, 5946, 7332, 7333, 7338, 7339, 7395]:
        return 'Photo and Video'
    if data['merchant_mcc'] in [5814]:
        return 'Fastfood'
    if data['merchant_mcc'] in [5193, 5992]:
        return 'Flowers'
    if data['merchant_mcc'] in [4812, 4813, 4814, 4816, 4829, 4900]:
        return 'Utility Services'
    if data['merchant_mcc'] in [ 6012, 6050, 6051, 6211, 6529, 6530, 6531, 
                                6532, 6533, 6534, 6536, 6537, 6538, 6540]:
        return 'Finances'
    if data['merchant_mcc'] in [7299, 7311, 7372, 7399, 7995, 8999]:
        return 'Business Services'

transactions['mcc'] = transactions['merchant_mcc']   
transactions['mcc'] = transactions.apply(lambda data : process_trans(data), axis=1)
transactions

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc,mcc
0,855115,7,3,1500,4554547,5411,Supermarkets
1,997036,6,6,0,1657528,5411,Supermarkets
2,398237,5,24,2500,26375569,5813,Restaurants
3,997036,6,2,0,16304402,5411,Supermarkets
4,291636,7,25,0,1259505,5411,Supermarkets
...,...,...,...,...,...,...,...
3951110,153986,6,30,29500,92701772,5719,"House, Repair"
3951111,346407,6,18,500,120278116,4816,Utility Services
3951112,358612,6,6,2500,111686898,5691,Clothing and shoes
3951113,346407,6,7,500,2765037,5411,Supermarkets


In [11]:
transactions.mcc.value_counts()

Supermarkets          1556219
Fastfood               547216
Transport              360585
Restaurants            260129
Fuel                   178480
Pharmacy               122483
Clothing and shoes     111315
House, Repair           93490
Business Services       51618
Utility Services        49325
Beauty                  44267
Art, Cinema, Music      43901
Auto Service            34530
Entertainments          29713
Car Rental              20293
Sporting goods          20292
Train Tickets           17242
Animals                 14242
Finances                12688
Flowers                 11090
Souvenirs                6329
Duty Free                3273
Photo and Video          2424
Name: mcc, dtype: int64

In [12]:
transactions

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc,mcc
0,855115,7,3,1500,4554547,5411,Supermarkets
1,997036,6,6,0,1657528,5411,Supermarkets
2,398237,5,24,2500,26375569,5813,Restaurants
3,997036,6,2,0,16304402,5411,Supermarkets
4,291636,7,25,0,1259505,5411,Supermarkets
...,...,...,...,...,...,...,...
3951110,153986,6,30,29500,92701772,5719,"House, Repair"
3951111,346407,6,18,500,120278116,4816,Utility Services
3951112,358612,6,6,2500,111686898,5691,Clothing and shoes
3951113,346407,6,7,500,2765037,5411,Supermarkets


In [13]:
transactions.mcc.fillna("No info", inplace=True)
transactions

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc,mcc
0,855115,7,3,1500,4554547,5411,Supermarkets
1,997036,6,6,0,1657528,5411,Supermarkets
2,398237,5,24,2500,26375569,5813,Restaurants
3,997036,6,2,0,16304402,5411,Supermarkets
4,291636,7,25,0,1259505,5411,Supermarkets
...,...,...,...,...,...,...,...
3951110,153986,6,30,29500,92701772,5719,"House, Repair"
3951111,346407,6,18,500,120278116,4816,Utility Services
3951112,358612,6,6,2500,111686898,5691,Clothing and shoes
3951113,346407,6,7,500,2765037,5411,Supermarkets


In [14]:
tran_agg = transactions.groupby('customer_id').transaction_amt.agg(
    sum_transaction_amt  = 'sum',
    mean_transaction_amt  = 'mean',
    num_transaction_amt = 'count',
    std_transactions_amt = 'std'
    
)
tran_agg

,sum_transaction_amt,mean_transaction_amt,num_transaction_amt,std_transactions_amt
customer_id,,,,
15,2221000,22210.000000,100,94708.222313
73,1149500,3768.852459,305,12401.381250
91,423500,3588.983051,118,8171.409234
144,5000,1000.000000,5,1172.603940
150,0,0.000000,4,0.000000
...,...,...,...,...
1098683,30000,638.297872,47,942.372859
1099266,17000,1062.500000,16,1641.899307
1099662,9500,339.285714,28,770.444454


In [15]:
#data_amt = transactions.groupby('mcc').transform(lambda x: x / x.max())
#transactions['transaction_normalized'] = data_amt['transaction_amt']

In [16]:
transactions

,customer_id,transaction_month,transaction_day,transaction_amt,merchant_id,merchant_mcc,mcc
0,855115,7,3,1500,4554547,5411,Supermarkets
1,997036,6,6,0,1657528,5411,Supermarkets
2,398237,5,24,2500,26375569,5813,Restaurants
3,997036,6,2,0,16304402,5411,Supermarkets
4,291636,7,25,0,1259505,5411,Supermarkets
...,...,...,...,...,...,...,...
3951110,153986,6,30,29500,92701772,5719,"House, Repair"
3951111,346407,6,18,500,120278116,4816,Utility Services
3951112,358612,6,6,2500,111686898,5691,Clothing and shoes
3951113,346407,6,7,500,2765037,5411,Supermarkets


In [17]:
sparse_transactions = pd.pivot_table(transactions, values='transaction_amt', 
                                     index=['customer_id'], aggfunc='sum',
                     columns=['mcc'], fill_value = 0).add_suffix('_amt')
sparse_transactions.head()

mcc,Animals_amt,"Art, Cinema, Music_amt",Auto Service_amt,Beauty_amt,Business Services_amt,Car Rental_amt,Clothing and shoes_amt,Duty Free_amt,Entertainments_amt,Fastfood_amt,...,No info_amt,Pharmacy_amt,Photo and Video_amt,Restaurants_amt,Souvenirs_amt,Sporting goods_amt,Supermarkets_amt,Train Tickets_amt,Transport_amt,Utility Services_amt
customer_id,,,,,,,,,,,,,,,,,,,,,
15,0,1000,0,8500,0,0,247500,2000,39000,50500,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
73,0,3500,0,0,8000,24000,61500,1000,68000,6000,...,597000,12500,0,211000,0,0,63000,0,53000,14000
91,0,0,18500,0,0,0,0,0,3000,6500,...,146500,0,0,19500,3500,0,5000,22500,21500,2000
144,0,0,0,0,0,0,0,0,0,0,...,0,3000,0,0,0,0,500,0,0,0
150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
full_transactions = tran_agg.merge(sparse_transactions, on = 'customer_id')
full_transactions.head(50)

,sum_transaction_amt,mean_transaction_amt,num_transaction_amt,std_transactions_amt,Animals_amt,"Art, Cinema, Music_amt",Auto Service_amt,Beauty_amt,Business Services_amt,Car Rental_amt,...,No info_amt,Pharmacy_amt,Photo and Video_amt,Restaurants_amt,Souvenirs_amt,Sporting goods_amt,Supermarkets_amt,Train Tickets_amt,Transport_amt,Utility Services_amt
customer_id,,,,,,,,,,,,,,,,,,,,,
15,2221000,22210.000000,100,94708.222313,0,1000,0,8500,0,0,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
73,1149500,3768.852459,305,12401.381250,0,3500,0,0,8000,24000,...,597000,12500,0,211000,0,0,63000,0,53000,14000
91,423500,3588.983051,118,8171.409234,0,0,18500,0,0,0,...,146500,0,0,19500,3500,0,5000,22500,21500,2000
144,5000,1000.000000,5,1172.603940,0,0,0,0,0,0,...,0,3000,0,0,0,0,500,0,0,0
150,0,0.000000,4,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,1000,1000.000000,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,187000,647.058824,289,2792.599347,0,0,0,27000,0,0,...,57500,5000,0,4000,0,2500,27000,0,29000,1500
183,15000,2500.000000,6,2366.431913,0,0,0,0,0,0,...,1000,0,0,1000,0,0,6500,0,0,0
192,1500,750.000000,2,353.553391,0,0,0,0,0,0,...,0,0,0,0,0,0,500,0,0,0


In [19]:
full_transactions.std_transactions_amt=full_transactions.std_transactions_amt.fillna(0)

3. Reactions processing

In [20]:
def label2id():
    return {
        'dislike': 1,
        'skip': 2, 
        'view': 3,
        'like': 4,
    }

def id2w():
    return {
        1: -10,
        2: -0.1,
        3: 0.1,
        4: 0.5,
    }

In [21]:
def process_reactions(reactions):
    reactions = reactions.copy()
    month = reactions['event_dttm'].dt.month
    month.name = 'month'
    weekday = reactions['event_dttm'].dt.weekday
    weekday.name = 'weekday'
    day = reactions['event_dttm'].dt.day
    day.name = 'day'
    hour = reactions['event_dttm'].dt.hour
    hour.name = 'hour'
    minute = reactions['event_dttm'].dt.minute
    minute.name = 'minute'
    is_weekend = weekday.between(5, 6).astype(int)
    is_weekend.name = 'is_weekend'
    
           
    new_reactions = pd.concat([reactions, month, weekday, day, hour, minute, is_weekend], axis=1)
    
    return new_reactions

stories_train = pd.read_csv('/kaggle/input/tinkoff/stories_reaction_train.csv', parse_dates=['event_dttm'])
train_reactions = process_reactions(stories_train)


stories_test = pd.read_csv('/kaggle/input/tinkoff/stories_reaction_test.csv', parse_dates=['event_dttm'])
test_reactions = process_reactions(stories_test)

In [22]:
train_agg =  train_reactions.groupby(['customer_id']).agg({'story_id': 'count'})
train_agg['story_count']=train_agg['story_id']
train_agg.drop('story_id', axis=1, inplace=True)
train_reactions = train_reactions.merge(train_agg, on=['customer_id'])
labelid = label2id()
train_reactions['event'] = train_reactions['event'].map(labelid)


test_agg =  test_reactions.groupby(['customer_id']).agg({'story_id': 'count'})
test_agg['story_count']=test_agg['story_id'] #amount of stories a user has seen
test_agg.drop('story_id', axis=1, inplace=True)
test_reactions = test_reactions.merge(test_agg, on=['customer_id'])

In [23]:
train_reactions

,customer_id,story_id,event_dttm,event,month,weekday,day,hour,minute,is_weekend,story_count
0,15,138,2018-07-24 15:33:22,3,7,1,24,15,33,0,49
1,15,202,2018-06-04 08:08:08,2,6,0,4,8,8,0,49
2,15,222,2018-06-17 13:44:45,2,6,6,17,13,44,1,49
3,15,379,2018-05-23 05:41:43,2,5,2,23,5,41,0,49
4,15,544,2018-07-25 02:16:29,3,7,2,25,2,16,0,49
...,...,...,...,...,...,...,...,...,...,...,...
473136,1103495,1220,2018-07-16 06:26:35,2,7,0,16,6,26,0,31
473137,1103495,1234,2018-07-15 19:44:38,3,7,6,15,19,44,1,31
473138,1103495,1234,2018-07-16 06:26:35,2,7,0,16,6,26,0,31
473139,1103495,1264,2018-07-15 19:44:38,3,7,6,15,19,44,1,31


In [24]:
#We have stories published in Agust only in the test set and stories published in March-July in the train set.
#We can conclude that there is overlapping between stories posting and customers transations in May-July
#but there are no transactions in August to predict user's reaction with. How reliable will be using 
#transactions information from the previous months?

test_reactions.month.max()   #8
test_reactions.month.min()   #8
train_reactions.month.min()  #3
train_reactions.month.max()  #7

7

In [25]:
train_data = train_reactions.merge(cus_train, how='left', on='customer_id')
train_data = train_data.merge(full_transactions, on='customer_id')

test_data = test_reactions.merge(cus_test, how='left', on='customer_id')
test_data = test_data.merge(full_transactions, how='left', on='customer_id')

In [26]:
train_data

,customer_id,story_id,event_dttm,event,month,weekday,day,hour,minute,is_weekend,...,No info_amt,Pharmacy_amt,Photo and Video_amt,Restaurants_amt,Souvenirs_amt,Sporting goods_amt,Supermarkets_amt,Train Tickets_amt,Transport_amt,Utility Services_amt
0,15,138,2018-07-24 15:33:22,3,7,1,24,15,33,0,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
1,15,202,2018-06-04 08:08:08,2,6,0,4,8,8,0,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
2,15,222,2018-06-17 13:44:45,2,6,6,17,13,44,1,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
3,15,379,2018-05-23 05:41:43,2,5,2,23,5,41,0,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
4,15,544,2018-07-25 02:16:29,3,7,2,25,2,16,0,...,1580500,8000,0,27500,2500,33000,198500,0,15000,2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448001,1103495,1220,2018-07-16 06:26:35,2,7,0,16,6,26,0,...,35000,0,0,0,0,0,12500,0,0,0
448002,1103495,1234,2018-07-15 19:44:38,3,7,6,15,19,44,1,...,35000,0,0,0,0,0,12500,0,0,0
448003,1103495,1234,2018-07-16 06:26:35,2,7,0,16,6,26,0,...,35000,0,0,0,0,0,12500,0,0,0
448004,1103495,1264,2018-07-15 19:44:38,3,7,6,15,19,44,1,...,35000,0,0,0,0,0,12500,0,0,0


In [27]:
test_data.isnull().sum()

customer_id                       0
story_id                          0
event_dttm                        0
answer_id                         0
month                             0
weekday                           0
day                               0
hour                              0
minute                            0
is_weekend                        0
story_count                       0
product_0                         1
product_1                         1
product_2                         1
product_3                         1
product_4                         1
product_5                         1
product_6                         1
gender_cd                         1
age                               1
marital_status_cd                 1
children_cnt                      1
first_session_dttm                1
job_position_cd                   1
first_ses                         1
months_after_first_session        1
sum_transaction_amt           11622
mean_transaction_amt        

In [28]:
test_data

,customer_id,story_id,event_dttm,answer_id,month,weekday,day,hour,minute,is_weekend,...,No info_amt,Pharmacy_amt,Photo and Video_amt,Restaurants_amt,Souvenirs_amt,Sporting goods_amt,Supermarkets_amt,Train Tickets_amt,Transport_amt,Utility Services_amt
0,466906,1152,2018-08-01 00:00:46,0,8,2,1,0,0,0,...,48500.0,1000.0,0.0,500.0,0.0,7000.0,14000.0,0.0,500.0,2500.0
1,466906,1355,2018-08-01 00:00:46,1,8,2,1,0,0,0,...,48500.0,1000.0,0.0,500.0,0.0,7000.0,14000.0,0.0,500.0,2500.0
2,146395,537,2018-08-01 00:02:08,2,8,2,1,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,146395,1368,2018-08-14 03:11:59,108851,8,1,14,3,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,897303,915,2018-08-01 00:02:30,3,8,2,1,0,2,0,...,2000.0,0.0,0.0,0.0,0.0,0.0,1500.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172044,621975,1247,2018-08-22 20:49:33,172044,8,2,22,20,49,0,...,1000.0,500.0,0.0,0.0,0.0,0.0,5500.0,0.0,0.0,0.0
172045,621975,1306,2018-08-22 20:49:33,172045,8,2,22,20,49,0,...,1000.0,500.0,0.0,0.0,0.0,0.0,5500.0,0.0,0.0,0.0
172046,621975,1537,2018-08-22 20:49:33,172046,8,2,22,20,49,0,...,1000.0,500.0,0.0,0.0,0.0,0.0,5500.0,0.0,0.0,0.0
172047,621975,836,2018-08-22 20:49:33,172047,8,2,22,20,49,0,...,1000.0,500.0,0.0,0.0,0.0,0.0,5500.0,0.0,0.0,0.0


In [29]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [30]:
cus_train.to_csv('train_customers.csv', index=False)
cus_test.to_csv('test_customers.csv', index=False)
train_reactions.to_csv('train_reactions.csv', index=False)
test_reactions.to_csv('test_stories_reactions.csv', index=False)

full_transactions.to_csv('full_sparse_transactions.csv', index=False)
transactions.to_csv('transactions.csv', index=False)